# Day16_2: DNN (Deep Neural Network) - 심층 신경망

## 학습 목표

**Part 1: 기초**
1. 깊은 신경망의 장점과 문제점 이해하기
2. 가중치 초기화 (Xavier, He) 이해하기
3. Batch Normalization 이해하기
4. Dropout으로 과적합 방지하기
5. 조기 종료(Early Stopping) 구현하기

**Part 2: 심화**
1. Learning Rate Scheduler 활용하기
2. 모델 저장과 로드 (torch.save/load)
3. 체크포인트 관리하기
4. Fashion-MNIST로 DNN 실습하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 깊은 신경망 | 복잡한 패턴 학습 | 이미지 인식, 자연어 처리 |
| 가중치 초기화 | 학습 안정성 확보 | 기울기 소실/폭발 방지 |
| BatchNorm | 빠른 학습, 일반화 | 층이 깊어도 안정적 학습 |
| Dropout | 과적합 방지 | 더 강건한 모델 |
| 체크포인트 | 모델 관리, 재개 | 긴 학습 과정 관리 |

**분석가 관점**: DNN은 더 깊은 층을 쌓아 복잡한 패턴을 학습합니다. 하지만 깊어질수록 학습이 어려워지는 문제가 있어, 이를 해결하는 다양한 기법을 배웁니다!

---

## 환경 설정

In [1]:
# 필수 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 고정
torch.manual_seed(42)
np.random.seed(42)

# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch 버전: {torch.__version__}")
print(f"사용 device: {device}")

PyTorch 버전: 2.9.0
사용 device: cpu


---

# Part 1: 기초

---

## 1.1 깊은 신경망의 장점과 문제점

### 층이 많으면 좋은가?

**장점: 더 복잡한 패턴 학습 가능**

```
얕은 네트워크 (2층):    입력 -> 은닉 -> 출력
깊은 네트워크 (5층):    입력 -> 은닉1 -> 은닉2 -> 은닉3 -> 은닉4 -> 출력
```

- 각 층이 점점 더 추상적인 특성을 학습
- 예: 이미지에서 Edge -> Texture -> Pattern -> Object

**문제점: 기울기 소실/폭발 (Gradient Vanishing/Exploding)**

- **기울기 소실**: 역전파 시 기울기가 점점 작아져 앞쪽 층이 학습되지 않음
- **기울기 폭발**: 기울기가 너무 커져서 학습이 불안정해짐

In [2]:
# 기울기 소실 문제 시뮬레이션
# Sigmoid 활성화 함수를 여러 층 통과하면 기울기가 어떻게 변하는지 확인

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)  # 최대값 0.25

# 10개 층을 통과할 때 기울기 변화
num_layers = 10
gradients = [1.0]  # 초기 기울기

for i in range(num_layers):
    # Sigmoid 도함수의 최대값(0.25)으로 계산
    gradients.append(gradients[-1] * 0.25)

print("층별 기울기 크기 (Sigmoid):")
for i, g in enumerate(gradients):
    print(f"  층 {i}: {g:.10f}")

print(f"\n10층 후 기울기: {gradients[-1]:.2e} (거의 0!)")

층별 기울기 크기 (Sigmoid):
  층 0: 1.0000000000
  층 1: 0.2500000000
  층 2: 0.0625000000
  층 3: 0.0156250000
  층 4: 0.0039062500
  층 5: 0.0009765625
  층 6: 0.0002441406
  층 7: 0.0000610352
  층 8: 0.0000152588
  층 9: 0.0000038147
  층 10: 0.0000009537

10층 후 기울기: 9.54e-07 (거의 0!)


In [3]:
# 기울기 소실 시각화
layers = list(range(len(gradients)))

fig = px.line(x=layers, y=gradients, markers=True,
              title='기울기 소실 문제 (Sigmoid 활성화)',
              labels={'x': '층 번호', 'y': '기울기 크기'})
fig.update_layout(yaxis_type='log')
fig.show()

### ReLU로 기울기 소실 완화

**ReLU (Rectified Linear Unit)**: `f(x) = max(0, x)`
- 양수 영역에서 도함수가 1 -> 기울기 유지
- 계산 효율적

In [4]:
# ReLU vs Sigmoid 비교
x = np.linspace(-5, 5, 100)

fig = make_subplots(rows=1, cols=2, subplot_titles=['활성화 함수', '도함수'])

# 활성화 함수
fig.add_trace(go.Scatter(x=x, y=sigmoid(x), name='Sigmoid'), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=np.maximum(0, x), name='ReLU'), row=1, col=1)

# 도함수
fig.add_trace(go.Scatter(x=x, y=sigmoid_derivative(x), name='Sigmoid\''), row=1, col=2)
fig.add_trace(go.Scatter(x=x, y=(x > 0).astype(float), name='ReLU\''), row=1, col=2)

fig.update_layout(title='Sigmoid vs ReLU', height=400)
fig.show()

---

## 1.2 가중치 초기화 (Weight Initialization)

### 왜 중요한가?

- **너무 작은 초기화**: 신호가 층을 지나며 점점 작아짐 (소실)
- **너무 큰 초기화**: 신호가 층을 지나며 점점 커짐 (폭발)
- **적절한 초기화**: 각 층의 출력 분산을 일정하게 유지

### Xavier (Glorot) 초기화

- **대상**: Sigmoid, Tanh 활성화 함수
- **공식**: `W ~ U(-sqrt(6/(fan_in + fan_out)), sqrt(6/(fan_in + fan_out)))`

### He (Kaiming) 초기화

- **대상**: ReLU 활성화 함수
- **공식**: `W ~ N(0, sqrt(2/fan_in))`

In [5]:
# PyTorch 기본 초기화 확인
layer = nn.Linear(100, 50)

print("PyTorch Linear 기본 초기화:")
print(f"  weight 평균: {layer.weight.data.mean():.6f}")
print(f"  weight 표준편차: {layer.weight.data.std():.6f}")
print(f"  weight 범위: [{layer.weight.data.min():.4f}, {layer.weight.data.max():.4f}]")

PyTorch Linear 기본 초기화:
  weight 평균: -0.000621
  weight 표준편차: 0.057475
  weight 범위: [-0.0998, 0.1000]


In [6]:
# Xavier 초기화 적용
layer_xavier = nn.Linear(100, 50)
nn.init.xavier_uniform_(layer_xavier.weight)

print("Xavier 초기화:")
print(f"  weight 평균: {layer_xavier.weight.data.mean():.6f}")
print(f"  weight 표준편차: {layer_xavier.weight.data.std():.6f}")

# He 초기화 적용
layer_he = nn.Linear(100, 50)
nn.init.kaiming_uniform_(layer_he.weight, nonlinearity='relu')

print("\nHe (Kaiming) 초기화:")
print(f"  weight 평균: {layer_he.weight.data.mean():.6f}")
print(f"  weight 표준편차: {layer_he.weight.data.std():.6f}")

Xavier 초기화:
  weight 평균: -0.001783
  weight 표준편차: 0.115465

He (Kaiming) 초기화:
  weight 평균: 0.003452
  weight 표준편차: 0.140310


In [7]:
# 초기화 방법에 따른 가중치 분포 비교
fig = make_subplots(rows=1, cols=3, subplot_titles=['기본 초기화', 'Xavier', 'He (Kaiming)'])

fig.add_trace(go.Histogram(x=layer.weight.data.numpy().flatten(), nbinsx=50, name='기본'), row=1, col=1)
fig.add_trace(go.Histogram(x=layer_xavier.weight.data.numpy().flatten(), nbinsx=50, name='Xavier'), row=1, col=2)
fig.add_trace(go.Histogram(x=layer_he.weight.data.numpy().flatten(), nbinsx=50, name='He'), row=1, col=3)

fig.update_layout(title='가중치 초기화 방법 비교', height=400, showlegend=False)
fig.show()

In [8]:
# 커스텀 초기화가 적용된 신경망
class DNNWithInitialization(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
        
        # He 초기화 적용
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.network(x)

# 모델 생성
model_init = DNNWithInitialization(784, [256, 128, 64], 10)
print(model_init)

DNNWithInitialization(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)


### 실무 예시: 언제 어떤 초기화를 사용하나요?

| 활성화 함수 | 추천 초기화 | PyTorch 함수 |
|------------|------------|-------------|
| Sigmoid, Tanh | Xavier | `nn.init.xavier_uniform_()` |
| ReLU, LeakyReLU | He (Kaiming) | `nn.init.kaiming_uniform_()` |
| SELU | LeCun | `nn.init.lecun_normal_()` |

---

## 1.3 Batch Normalization

### 내부 공변량 이동 (Internal Covariate Shift)

학습 중 각 층의 입력 분포가 계속 변해서 학습이 어려워지는 현상입니다.

### Batch Normalization의 효과

1. **정규화**: 미니배치의 평균과 분산으로 정규화
2. **스케일/시프트**: 학습 가능한 파라미터로 조정

$$\hat{x} = \frac{x - \mu_{batch}}{\sqrt{\sigma^2_{batch} + \epsilon}}$$
$$y = \gamma \hat{x} + \beta$$

In [9]:
# BatchNorm1d 사용법
bn = nn.BatchNorm1d(num_features=64)  # 특성 수

# 배치 데이터 생성
x = torch.randn(32, 64)  # batch_size=32, features=64

# BatchNorm 적용 전
print("BatchNorm 적용 전:")
print(f"  평균: {x.mean():.4f}, 표준편차: {x.std():.4f}")

# BatchNorm 적용 후
bn.train()  # 학습 모드
x_bn = bn(x)
print("\nBatchNorm 적용 후:")
print(f"  평균: {x_bn.mean():.4f}, 표준편차: {x_bn.std():.4f}")

BatchNorm 적용 전:
  평균: -0.0188, 표준편차: 0.9927

BatchNorm 적용 후:
  평균: -0.0000, 표준편차: 1.0002


In [10]:
# BatchNorm 학습 파라미터 확인
print("BatchNorm 학습 가능 파라미터:")
print(f"  gamma (weight): shape={bn.weight.shape}")
print(f"  beta (bias): shape={bn.bias.shape}")

print("\nBatchNorm 통계량 (학습 중 업데이트):")
print(f"  running_mean: shape={bn.running_mean.shape}")
print(f"  running_var: shape={bn.running_var.shape}")

BatchNorm 학습 가능 파라미터:
  gamma (weight): shape=torch.Size([64])
  beta (bias): shape=torch.Size([64])

BatchNorm 통계량 (학습 중 업데이트):
  running_mean: shape=torch.Size([64])
  running_var: shape=torch.Size([64])


In [11]:
# BatchNorm이 포함된 DNN
class DNNWithBatchNorm(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))  # BatchNorm 추가
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

model_bn = DNNWithBatchNorm(784, [256, 128, 64], 10)
print(model_bn)

DNNWithBatchNorm(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)


### 실무 팁: BatchNorm 순서

**권장 순서**: `Linear -> BatchNorm -> ReLU`

- BatchNorm이 활성화 함수 전에 위치
- 일부는 `Linear -> ReLU -> BatchNorm` 사용 (결과 비슷)

---

## 1.4 Dropout

### 과적합 방지 원리

**학습 시**: 무작위로 뉴런을 비활성화 (p 확률로 0으로 설정)
**추론 시**: 모든 뉴런 사용 (출력을 1-p로 스케일링)

```
학습 시:    [1] [0] [1] [1] [0] [1]   (일부 비활성화)
추론 시:    [1] [1] [1] [1] [1] [1]   (전체 사용)
```

In [12]:
# Dropout 사용법
dropout = nn.Dropout(p=0.5)  # 50% 비활성화

x = torch.ones(10)
print(f"원본: {x}")

# 학습 모드
dropout.train()
print(f"학습 모드 (dropout): {dropout(x)}")
print(f"학습 모드 (dropout): {dropout(x)}  # 매번 다름")

# 평가 모드
dropout.eval()
print(f"평가 모드: {dropout(x)}  # 변화 없음")

원본: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
학습 모드 (dropout): tensor([0., 2., 0., 0., 0., 0., 2., 0., 0., 2.])
학습 모드 (dropout): tensor([0., 2., 0., 2., 2., 2., 0., 0., 2., 0.])  # 매번 다름
평가 모드: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])  # 변화 없음


In [13]:
# Dropout이 포함된 DNN
class DNNWithDropout(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate=0.5):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))  # Dropout 추가
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

model_dropout = DNNWithDropout(784, [256, 128, 64], 10, dropout_rate=0.3)
print(model_dropout)

DNNWithDropout(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=10, bias=True)
  )
)


### 실무 팁: Dropout 비율 선택

| 레이어 위치 | 권장 비율 |
|------------|----------|
| 입력층 근처 | 0.2 |
| 은닉층 | 0.3 ~ 0.5 |
| 출력층 근처 | 0.5 이하 |

---

## 1.5 조기 종료 (Early Stopping)

### 언제 학습을 멈출까?

검증 손실이 더 이상 개선되지 않으면 학습을 중단합니다.

```
학습 손실:  \___     (계속 감소)
검증 손실:  \__/---  (감소 후 증가 = 과적합)
                ^
             여기서 멈춤!
```

In [14]:
# Early Stopping 클래스 구현
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        """
        patience: 개선 없이 기다릴 에포크 수
        min_delta: 개선으로 인정할 최소 변화량
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            print(f"  EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
        
        return self.early_stop

# 사용 예시
early_stopper = EarlyStopping(patience=3)

# 가상의 검증 손실 시퀀스
val_losses = [0.5, 0.4, 0.35, 0.36, 0.37, 0.38, 0.39]

for epoch, loss in enumerate(val_losses):
    print(f"Epoch {epoch}: val_loss = {loss}")
    if early_stopper(loss):
        print(f"Early stopping at epoch {epoch}!")
        break

Epoch 0: val_loss = 0.5
Epoch 1: val_loss = 0.4
Epoch 2: val_loss = 0.35
Epoch 3: val_loss = 0.36
  EarlyStopping counter: 1/3
Epoch 4: val_loss = 0.37
  EarlyStopping counter: 2/3
Epoch 5: val_loss = 0.38
  EarlyStopping counter: 3/3
Early stopping at epoch 5!


---

# Part 2: 심화

---

## 2.1 Learning Rate Scheduler

학습률을 동적으로 조정하여 학습을 최적화합니다.

### 주요 스케줄러

1. **StepLR**: 일정 간격마다 학습률 감소
2. **ReduceLROnPlateau**: 손실이 정체되면 감소
3. **CosineAnnealingLR**: 코사인 함수로 감소

In [15]:
# 다양한 스케줄러 비교
def simulate_scheduler(scheduler_class, scheduler_params, epochs=50):
    """스케줄러의 학습률 변화를 시뮬레이션"""
    model = nn.Linear(10, 1)
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    scheduler = scheduler_class(optimizer, **scheduler_params)
    
    lrs = []
    for epoch in range(epochs):
        lrs.append(optimizer.param_groups[0]['lr'])
        scheduler.step()
    
    return lrs

# 각 스케줄러 시뮬레이션
epochs = 50

lrs_step = simulate_scheduler(optim.lr_scheduler.StepLR, {'step_size': 10, 'gamma': 0.5}, epochs)
lrs_exp = simulate_scheduler(optim.lr_scheduler.ExponentialLR, {'gamma': 0.95}, epochs)
lrs_cosine = simulate_scheduler(optim.lr_scheduler.CosineAnnealingLR, {'T_max': epochs}, epochs)

# 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(y=lrs_step, name='StepLR (step=10, gamma=0.5)'))
fig.add_trace(go.Scatter(y=lrs_exp, name='ExponentialLR (gamma=0.95)'))
fig.add_trace(go.Scatter(y=lrs_cosine, name='CosineAnnealingLR'))

fig.update_layout(title='Learning Rate Scheduler 비교',
                  xaxis_title='Epoch',
                  yaxis_title='Learning Rate')
fig.show()

In [16]:
# ReduceLROnPlateau 사용 예시
model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 손실이 3 에포크 동안 개선 없으면 학습률 0.1배로 감소
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',      # 손실 최소화
    factor=0.1,      # 학습률 * 0.1
    patience=3,      # 3 에포크 기다림
    verbose=True     # 로그 출력
)

# 사용법: scheduler.step(val_loss)
print(f"ReduceLROnPlateau 설정 완료")
print(f"  mode: min (손실 최소화)")
print(f"  factor: 0.1 (학습률 * 0.1)")
print(f"  patience: 3 (3 에포크 기다림)")

TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

---

## 2.2 모델 저장과 로드

### state_dict 개념

모델의 학습 가능한 파라미터(가중치, 편향)를 딕셔너리 형태로 저장합니다.

In [ ]:
# 모델 생성
model = DNNWithBatchNorm(784, [256, 128], 10)

# state_dict 확인
print("Model state_dict 키:")
for key in model.state_dict().keys():
    print(f"  {key}")

In [ ]:
# 방법 1: state_dict만 저장 (권장)
torch.save(model.state_dict(), 'model_weights.pth')
print("state_dict 저장 완료: model_weights.pth")

# 로드
model_loaded = DNNWithBatchNorm(784, [256, 128], 10)  # 같은 구조로 생성
model_loaded.load_state_dict(torch.load('model_weights.pth'))
model_loaded.eval()  # 평가 모드 설정
print("state_dict 로드 완료!")

In [ ]:
# 방법 2: 전체 모델 저장
torch.save(model, 'full_model.pth')
print("전체 모델 저장 완료: full_model.pth")

# 로드
model_full = torch.load('full_model.pth')
model_full.eval()
print("전체 모델 로드 완료!")

### 실무 팁: state_dict vs 전체 모델

| 방법 | 장점 | 단점 |
|------|------|------|
| state_dict | 유연함, 호환성 좋음 | 모델 구조 별도 정의 필요 |
| 전체 모델 | 간편함 | PyTorch 버전 의존성 |

---

## 2.3 체크포인트 관리

학습 중 최고 성능 모델을 저장하고, 학습을 재개할 수 있도록 합니다.

In [ ]:
# 체크포인트 저장 함수
def save_checkpoint(model, optimizer, epoch, loss, path='checkpoint.pth'):
    """학습 상태 저장"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at epoch {epoch} (loss: {loss:.4f})")

# 체크포인트 로드 함수
def load_checkpoint(model, optimizer, path='checkpoint.pth'):
    """학습 상태 복원"""
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded from epoch {epoch} (loss: {loss:.4f})")
    return epoch, loss

In [ ]:
# 최고 성능 모델 저장 클래스
class ModelCheckpoint:
    def __init__(self, filepath='best_model.pth', verbose=True):
        self.filepath = filepath
        self.verbose = verbose
        self.best_loss = float('inf')
    
    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            if self.verbose:
                print(f"  Validation loss improved ({self.best_loss:.4f} -> {val_loss:.4f}). Saving model...")
            self.best_loss = val_loss
            torch.save(model.state_dict(), self.filepath)
            return True
        return False

# 사용 예시
model_ckpt = ModelCheckpoint('best_model.pth')
print(f"ModelCheckpoint 설정 완료: {model_ckpt.filepath}")

---

## 2.4 Fashion-MNIST로 DNN 실습

Fashion-MNIST는 10개 클래스의 패션 아이템 이미지 데이터셋입니다.

| 레이블 | 클래스 |
|--------|--------|
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

In [ ]:
# Fashion-MNIST 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform
)

print(f"학습 데이터: {len(train_dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")
print(f"이미지 shape: {train_dataset[0][0].shape}")

In [ ]:
# 샘플 이미지 시각화
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

fig = make_subplots(rows=2, cols=5, subplot_titles=[class_names[i] for i in range(10)])

for i in range(10):
    # 각 클래스의 첫 번째 이미지 찾기
    for img, label in train_dataset:
        if label == i:
            row = i // 5 + 1
            col = i % 5 + 1
            fig.add_trace(
                go.Heatmap(z=img.squeeze().numpy(), colorscale='gray', showscale=False),
                row=row, col=col
            )
            break

fig.update_layout(title='Fashion-MNIST 샘플', height=500)
fig.show()

In [ ]:
# DataLoader 생성
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"학습 배치 수: {len(train_loader)}")
print(f"테스트 배치 수: {len(test_loader)}")

In [ ]:
# 완전한 DNN 모델 정의
class FashionDNN(nn.Module):
    def __init__(self, input_dim=784, hidden_dims=[256, 128, 64], output_dim=10, dropout_rate=0.3):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten: (batch, 1, 28, 28) -> (batch, 784)
        return self.network(x)

# 모델 생성
model = FashionDNN()
model = model.to(device)
print(model)

# 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n총 파라미터: {total_params:,}")
print(f"학습 가능 파라미터: {trainable_params:,}")

In [ ]:
# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Early Stopping & Checkpoint
early_stopper = EarlyStopping(patience=5)
model_ckpt = ModelCheckpoint('fashion_best_model.pth')

print("학습 설정 완료!")

In [ ]:
# 학습 함수
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(train_loader), 100. * correct / total

# 평가 함수
def evaluate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(test_loader), 100. * correct / total

In [ ]:
# 학습 실행
epochs = 30
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}

print("학습 시작!")
print("=" * 60)

for epoch in range(epochs):
    # 학습
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 검증
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)
    
    # 학습률
    current_lr = optimizer.param_groups[0]['lr']
    
    # 기록
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    print(f"Epoch {epoch+1:3d}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | LR: {current_lr:.6f}")
    
    # 스케줄러 업데이트
    scheduler.step(val_loss)
    
    # 체크포인트 저장
    model_ckpt(val_loss, model)
    
    # Early Stopping 체크
    if early_stopper(val_loss):
        print(f"\nEarly stopping at epoch {epoch+1}!")
        break

print("\n학습 완료!")

In [ ]:
# 학습 곡선 시각화
fig = make_subplots(rows=1, cols=3, subplot_titles=['Loss', 'Accuracy', 'Learning Rate'])

# Loss
fig.add_trace(go.Scatter(y=history['train_loss'], name='Train Loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=history['val_loss'], name='Val Loss'), row=1, col=1)

# Accuracy
fig.add_trace(go.Scatter(y=history['train_acc'], name='Train Acc'), row=1, col=2)
fig.add_trace(go.Scatter(y=history['val_acc'], name='Val Acc'), row=1, col=2)

# Learning Rate
fig.add_trace(go.Scatter(y=history['lr'], name='LR'), row=1, col=3)

fig.update_layout(title='Fashion-MNIST DNN 학습 곡선', height=400)
fig.show()

In [ ]:
# 최고 성능 모델 로드
model.load_state_dict(torch.load('fashion_best_model.pth'))
model.eval()

# 최종 평가
final_loss, final_acc = evaluate(model, test_loader, criterion, device)
print(f"최고 성능 모델 테스트 정확도: {final_acc:.2f}%")

In [ ]:
# 혼동 행렬 계산
from sklearn.metrics import confusion_matrix

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

cm = confusion_matrix(all_labels, all_preds)

# 혼동 행렬 시각화
fig = px.imshow(cm, labels=dict(x="Predicted", y="Actual", color="Count"),
                x=class_names, y=class_names,
                title='Fashion-MNIST 혼동 행렬',
                color_continuous_scale='Blues',
                text_auto=True)
fig.show()

In [ ]:
# 클래스별 정확도
class_correct = np.diag(cm)
class_total = cm.sum(axis=1)
class_accuracy = class_correct / class_total * 100

class_acc_df = pd.DataFrame({
    'Class': class_names,
    'Accuracy': class_accuracy
}).sort_values('Accuracy', ascending=True)

fig = px.bar(class_acc_df, x='Accuracy', y='Class', orientation='h',
             title='클래스별 정확도',
             color='Accuracy', color_continuous_scale='RdYlGn')
fig.show()

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. 가중치 초기화 (기본)

**문제**: `nn.Linear(100, 50)` 레이어를 생성하고 He 초기화를 적용하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q2. BatchNorm 적용 (기본)

**문제**: 32개의 특성을 가진 배치 데이터에 BatchNorm1d를 적용하고, 적용 전후의 평균과 표준편차를 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. Dropout 동작 확인 (기본)

**문제**: `p=0.5`인 Dropout 레이어를 만들고, 학습 모드와 평가 모드에서의 차이를 확인하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. Early Stopping 구현 (응용)

**문제**: patience=3인 Early Stopping 클래스를 사용하여, 아래 검증 손실 시퀀스에서 언제 학습이 중단되는지 확인하세요.

```python
val_losses = [0.8, 0.6, 0.5, 0.48, 0.49, 0.50, 0.51]
```

In [ ]:
# 여기에 코드를 작성하세요


### Q5. Learning Rate Scheduler (응용)

**문제**: StepLR 스케줄러를 사용하여 10 에포크마다 학습률을 0.1배로 감소시키세요. 30 에포크 동안의 학습률 변화를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 모델 저장/로드 (응용)

**문제**: 간단한 MLP 모델을 생성하고, state_dict를 저장한 뒤 새로운 모델에 로드하세요. 두 모델의 첫 번째 레이어 가중치가 동일한지 확인하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 체크포인트 저장 (복합)

**문제**: 모델, 옵티마이저, 에포크, 손실을 포함한 체크포인트를 저장하고 로드하는 함수를 구현하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 과적합 진단 (복합)

**문제**: 학습 손실과 검증 손실 곡선을 보고 과적합 여부를 판단하세요. 아래 데이터를 Plotly로 시각화하고, 과적합이 시작되는 에포크를 찾으세요.

```python
train_losses = [2.0, 1.5, 1.0, 0.7, 0.5, 0.3, 0.2, 0.15, 0.1, 0.08]
val_losses = [2.1, 1.6, 1.1, 0.8, 0.7, 0.75, 0.8, 0.9, 1.0, 1.1]
```

In [ ]:
# 여기에 코드를 작성하세요


### Q9. Fashion-MNIST DNN (종합)

**문제**: 다음 구조의 DNN을 정의하고 Fashion-MNIST에서 10 에포크 학습하세요.

- 입력: 784
- 은닉층: 512 -> 256 -> 128
- 출력: 10
- BatchNorm, Dropout(0.4), ReLU 사용

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합 파이프라인 (종합)

**문제**: Fashion-MNIST 학습 파이프라인을 완성하세요. 다음 요소를 모두 포함해야 합니다.

1. DNN 모델 (He 초기화, BatchNorm, Dropout)
2. Adam 옵티마이저
3. ReduceLROnPlateau 스케줄러
4. Early Stopping (patience=5)
5. 최고 모델 체크포인트 저장
6. 학습 곡선 시각화 (Plotly)

In [ ]:
# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 언제 사용? |
|-----|----------|----------|
| 가중치 초기화 | Xavier(Sigmoid/Tanh), He(ReLU) | 모든 신경망 |
| BatchNorm | 미니배치 정규화 | 깊은 네트워크, 빠른 수렴 |
| Dropout | 무작위 뉴런 비활성화 | 과적합 방지 |
| Early Stopping | 검증 손실 모니터링 | 자동 학습 종료 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 메서드 | 언제 사용? |
|-----|-----------|----------|
| LR Scheduler | StepLR, ReduceLROnPlateau | 학습률 동적 조정 |
| 모델 저장 | torch.save(state_dict) | 학습 결과 보존 |
| 체크포인트 | 모델+옵티마이저+에포크 | 학습 재개, 최고 모델 저장 |

### DNN 모델 구조 패턴

```python
# 권장 패턴
Linear -> BatchNorm -> ReLU -> Dropout
Linear -> BatchNorm -> ReLU -> Dropout
...
Linear (출력층)
```

### 실무 팁

1. **He 초기화**: ReLU 계열 활성화 함수와 함께 사용
2. **BatchNorm**: 학습 속도 향상, 더 높은 학습률 사용 가능
3. **Dropout**: 0.3~0.5 범위로 시작, 필요에 따라 조정
4. **Early Stopping**: patience=5~10 권장
5. **체크포인트**: 긴 학습 시 반드시 사용
6. **eval() 모드**: 평가 시 반드시 `model.eval()` 호출

In [ ]:
# 임시 파일 정리
import os

temp_files = ['model_weights.pth', 'full_model.pth', 'checkpoint.pth', 
              'best_model.pth', 'fashion_best_model.pth']

for f in temp_files:
    if os.path.exists(f):
        os.remove(f)
        print(f"삭제됨: {f}")